<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_metric_kernel_enhanced_fixed_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
quantum_metric_kernel_enhanced_fixed.py

Enhanced quantum-metric learning for 4-bit parity classification:

1) Analytic CNOT cascade baseline (100% accuracy)
2) Variational QNN baseline (5-fold CV)
3) Enhanced metric learning:
   - AngleEmbedding
   - Layer 1: Ry(params[0:n_bits])
   - CNOT ring entanglement
   - Layer 2: RZ(params[n_bits:2*n_bits]) + Ry(params[2*n_bits:3*n_bits])
   - Mini-batch centered alignment loss
   - QNGOptimizer with explicit metric tensor
4) Final SVM (precomputed kernel) 5-fold CV
5) Fidelity histogram of learned kernel
"""

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Hyperparameters
# -----------------------------------------------------------------------------
n_bits     = 4       # number of input bits/qubits
qnn_epochs = 30      # epochs for variational QNN
qnn_lr     = 0.4     # learning rate for QNN
n_steps    = 200     # training steps for metric learning
qng_lr     = 0.05    # QNG step size
svm_C      = 1.0     # SVM regularization parameter
batch_size = 8       # mini-batch size for alignment loss
seed       = 42      # random seed

np.random.seed(seed)
pnp.random.seed(seed)

# -----------------------------------------------------------------------------
# 1) Generate 4‐bit parity dataset
# -----------------------------------------------------------------------------
X_int    = pnp.array(list(itertools.product([0, 1], repeat=n_bits)), dtype=int)
y        = pnp.array(np.sum(X_int, axis=1) % 2, dtype=int)  # {0,1}
y_signed = 1 - 2 * y                                        # {+1,-1}
X_feat   = X_int.astype(float)                              # for AngleEmbedding

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# -----------------------------------------------------------------------------
# 2) Analytic CNOT cascade baseline
# -----------------------------------------------------------------------------
dev_exact = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_exact)
def analytic_circuit(x):
    for i, bit in enumerate(x):
        if bit:
            qml.PauliX(wires=i)
    for i in range(n_bits):
        qml.CNOT(wires=[i, n_bits])
    return qml.expval(qml.PauliZ(wires=n_bits))

preds_analytic = [0 if analytic_circuit(x) > 0 else 1 for x in X_int]
acc_analytic   = accuracy_score(y, preds_analytic)
print(f"Analytic CNOT Cascade Accuracy: {acc_analytic:.3f}")

# -----------------------------------------------------------------------------
# 3) Variational QNN baseline (5‐fold CV)
# -----------------------------------------------------------------------------
dev_qnn = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_qnn, interface="autograd")
def qnn_circuit(params, x):
    for i, bit in enumerate(x):
        if bit:
            qml.RY(np.pi, wires=i)
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    for i in range(n_bits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.expval(qml.PauliZ(wires=n_bits - 1))

var_scores = []
for tr, te in kf.split(X_int, y):
    params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
    opt    = qml.GradientDescentOptimizer(stepsize=qnn_lr)

    for _ in range(qnn_epochs):
        def cost(p):
            preds = [qnn_circuit(p, X_int[i]) for i in tr]
            return pnp.mean((pnp.array(preds) - y_signed[tr]) ** 2)
        params = opt.step(cost, params)

    preds_te = [qnn_circuit(params, X_int[i]) for i in te]
    bits     = [0 if v > 0 else 1 for v in preds_te]
    var_scores.append(accuracy_score(y[te], bits))

print(f"Variational QNN CV Accuracy: {np.mean(var_scores):.3f} ± {np.std(var_scores):.3f}")

# -----------------------------------------------------------------------------
# 4) Enhanced quantum‐metric learning
# -----------------------------------------------------------------------------
dev_feat = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_feat, interface="autograd")
def feature_map(params, x):
    # Angle embedding of data
    qml.templates.AngleEmbedding(x, wires=range(n_bits))
    # Layer 1: Ry(params[0:n_bits])
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    # Ring entanglement
    for i in range(n_bits):
        qml.CNOT(wires=[i, (i + 1) % n_bits])
    # Layer 2: RZ + Ry
    for i in range(n_bits):
        qml.RZ(params[n_bits + i], wires=i)
        qml.RY(params[2 * n_bits + i], wires=i)
    return qml.state()

def build_kernel_matrix(params, indices=None):
    Xp = X_feat if indices is None else X_feat[indices]
    states = pnp.stack([feature_map(params, x) for x in Xp])
    re, im = pnp.real(states), pnp.imag(states)
    real_ov = re @ re.T + im @ im.T
    imag_ov = re @ im.T - im @ re.T
    return real_ov**2 + imag_ov**2

def centered_alignment(params, indices):
    K   = build_kernel_matrix(params, indices)
    yi  = y_signed[indices]
    N   = len(indices)
    H   = pnp.eye(N) - pnp.ones((N, N)) / N
    Kc  = H @ K @ H
    T   = pnp.outer(yi, yi)
    num = pnp.sum(Kc * T)
    den = pnp.linalg.norm(Kc) * pnp.linalg.norm(T)
    return num / den

# Initialize metric‐learning parameters: 3*n_bits trainable angles
params_k = pnp.random.randn(3 * n_bits, requires_grad=True) * 0.1

# Explicit Fisher information function (argument 0 only)
raw_metric = qml.metric_tensor(feature_map, argnum=[0])

def metric_tensor_fn(params, indices):
    mats = []
    for idx in indices:
        m = raw_metric(params, X_feat[idx])
        if isinstance(m, tuple):
            m = m[0]
        mats.append(m)
    return sum(mats) / len(mats)

opt_qng = qml.QNGOptimizer(stepsize=qng_lr)

print("\nTraining enhanced feature map via stochastic QNG + alignment")
for step in range(n_steps):
    # sample a random mini‐batch
    batch = np.random.choice(len(X_feat), size=batch_size, replace=False)
    # define mini‐batch loss (negative centered alignment)
    loss_fn = lambda p: -centered_alignment(p, batch)
    # natural‐gradient step with per‐batch Fisher
    params_k = opt_qng.step(
        loss_fn,
        params_k,
        metric_tensor_fn=lambda p: metric_tensor_fn(p, batch),
    )
    if step % 20 == 0:
        full_align = centered_alignment(params_k, np.arange(len(X_feat)))
        print(f" Step {step:>3}: full alignment = {full_align:.4f}")

# -----------------------------------------------------------------------------
# 5) Final SVM (precomputed kernel) 5‐fold CV
# -----------------------------------------------------------------------------
K_final   = np.array(build_kernel_matrix(params_k))
ker_scores = []

for tr, te in kf.split(K_final, y):
    svc = SVC(kernel="precomputed", C=svm_C)
    svc.fit(K_final[np.ix_(tr, tr)], y[tr])
    preds = svc.predict(K_final[np.ix_(te, tr)])
    ker_scores.append(accuracy_score(y[te], preds))

print(
    f"\nEnhanced Quantum‐Metric Kernel CV Accuracy: "
    f"{np.mean(ker_scores):.3f} ± {np.std(ker_scores):.3f}"
)

# -----------------------------------------------------------------------------
# 6) Fidelity histogram
# -----------------------------------------------------------------------------
fids = K_final.flatten()
plt.figure(figsize=(6, 4))
plt.hist(fids, bins=30, color="C0", alpha=0.8)
plt.title("Fidelity Distribution of Learned Kernel")
plt.xlabel("Kernel value |⟨ψ_i|ψ_j⟩|²")
plt.ylabel("Count")
plt.tight_layout()
plt.show()